# Import timeframe data

In [1]:
import numpy as np


# Import data

In [2]:
games = np.load('/tmp/game_data.npy')
print (games.shape)

(49996, 6, 266)


# Look at data

In [3]:
#print(games[1][4])

In [4]:
labels = np.load('/tmp/game_labels.npy')
print (labels.shape)

(49996, 2)


# Separate train and test data

In [7]:

#labels_binary_length_corrected = labels.reshape((300000, 2))
labels_binary_length_corrected = labels

number_of_features_to_use = games.shape[2]

data = np.zeros((games.shape[0],games.shape[1], number_of_features_to_use))
for i in range(games.shape[0]):
    for j in range(games.shape[1]):
        data[i, j] = games[i,j][:number_of_features_to_use]
print (data.shape)


val_split = int(len(data) * 0.9)
train_split = int(val_split * 0.8)
print(data.shape)
x_train = data[:train_split,:]
y_train = labels_binary_length_corrected[:train_split,:]
x_val = data[train_split:val_split,:]
y_val = labels_binary_length_corrected[train_split:val_split,:]
x_test = data[val_split:,:]
y_test = labels_binary_length_corrected[val_split:,:]

#x_train = np.reshape(x_train, x_train.shape + (1,))
#x_test = np.reshape(x_test, x_test.shape + (1,))

print(x_train.shape)

(49996, 6, 266)
(49996, 6, 266)
(35996, 6, 266)


In [10]:
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.regularizers import l2
import keras
from sklearn.metrics import roc_auc_score


# batch eta lambda units

results = open('/tmp/dota_coarse_search.csv', 'a')
#results.write("val_acc, auc, val_acc_certain, auc_certain, batch_size, lstm_size, dense_size, regularization lstm, eta")

batch_sizes = [2**y for y in range(5,10)]

for i in range(1):

    batch_size = np.random.choice(batch_sizes)
    #print("batch size: ", batch_size)
    lstm_size = np.random.randint(low=50, high=1500)
    #print("lstm size: ", lstm_size)
    regu = np.random.uniform(low=0, high=0.1)
    #print("regularization: ", regu)
    eta = np.random.uniform(low=1e-6, high=0.01)
    #print("eta: ", eta)
    
    model = Sequential()

    model.add(LSTM(
        input_shape=(None, x_train.shape[2]),
        units=lstm_size,
        return_sequences=False,
        kernel_regularizer= l2(regu))
        )

    
    dense_size = np.random.randint(low=50,high=2000)
    #print("dense size: ", dense_size)
    model.add(Dense(
        units=dense_size))
    model.add(Activation('relu'))

    model.add(Dense(
        units=2))
    model.add(Activation('softmax'))
    #model.compile(loss='mse', optimizer='rmsprop')
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=eta), metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])
    
    model_checkpoint = keras.callbacks.ModelCheckpoint('/tmp/best_weights', monitor='val_categorical_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    history = model.fit(x_train, y_train, epochs=2, batch_size=batch_size, validation_split=0.0, validation_data=(x_val, y_val), verbose=2, callbacks = [model_checkpoint])
  
    model.load_weights('/tmp/best_weights')
    
    
    # Test loss and accuracy
    loss_and_metrics1 = model.evaluate(x_val, y_val, verbose=2)
    
    
    ## Calculate AUC
    y_pred = model.predict_proba(x_val, verbose=0)
    score1 = roc_auc_score(y_val, y_pred)

    ## Remove uncertain predictions
    ind1 = y_pred[:,0] > 0.6
    ind2 = y_pred[:,1] > 0.6
    ind = ind1 + ind2
    y_pred2 = y_pred[ind,:]
    y_val2 = y_val[ind,:]
    x_val2 = x_val[ind,:]
    #print(ind)
    #print(y_pred2.shape)
    #print(y_pred.shape)
    score2 = roc_auc_score(y_val2, y_pred2)
    #print(score2)

    loss_and_metrics2 = model.evaluate(x_val2, y_val2, verbose=2)
    #print(loss_and_metrics)
    
    log = ",".join( list(map(str, [loss_and_metrics1[2], score1, loss_and_metrics2[2], score2, batch_size, lstm_size, dense_size, regu, eta, ]))) + "\n"
    print(log)
    results.write(log)
    #print(loss_and_metrics)

results.close()

Train on 35996 samples, validate on 9000 samples
Epoch 1/2
10s - loss: 8.3914 - mean_absolute_error: 0.5171 - categorical_accuracy: 0.4828 - val_loss: 8.4691 - val_mean_absolute_error: 0.5227 - val_categorical_accuracy: 0.4773
Epoch 2/2
9s - loss: 8.3773 - mean_absolute_error: 0.5170 - categorical_accuracy: 0.4830 - val_loss: 8.4691 - val_mean_absolute_error: 0.5227 - val_categorical_accuracy: 0.4773
0.477333333333,0.503256136166,0.477333333333,0.503256136166,128,1134,1147,0.012043624687272181,0.0035089762767319655



In [ ]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# AUC score

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = model.predict_proba(x_test, verbose=0)
score = roc_auc_score(y_test, y_pred)
print(score)

# Remove unsure predictions

In [ ]:
y_pred = model.predict_proba(x_test, verbose=0)
ind1 = y_pred[:,0] > 0.6
ind2 = y_pred[:,1] > 0.6
ind = ind1 + ind2
y_pred2 = y_pred[ind]
y_test2 = y_test[ind]
x_test2 = x_test[ind]
print(ind)
print(y_pred2.shape)
print(y_pred.shape)
score = roc_auc_score(y_test2, y_pred2)
print(score)

loss_and_metrics = model.evaluate(x_test2, y_test2, verbose=2)
print(loss_and_metrics)